# Implicit ALS model epic 5

Deze notebook implementeert een hybrid recommender systeem met content-based en collaborative filtering. Data wordt geladen vanuit een materialized view en ondergaat eerst content-based filtering op basis van tekstkenmerken van campagnes. Daarna wordt collaborative filtering toegepast met het Alternating Least Squares (ALS) model. De hybrid aanpak combineert de resultaten van beide methoden om een lijst van top 20 gebruikers te genereren die mogelijk geïnteresseerd zijn in een specifieke campagne. 

Het ALS model werd gekozen omdat het beter omkan met sparse matrices en impliciete ratings dan meer conventionele modellen zoals SVD. Met de content-based filtering wordt er getracht om *cold start* problemen te vermijden.

De itembased.ipynb notebook implementeert SVD en wordt niet verder gebruikt.

## 1. Imports

In [22]:
import os
import sqlalchemy
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from collections import Counter
from scipy.sparse import coo_matrix
from sqlalchemy import create_engine, text
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight
from sentence_transformers import SentenceTransformer
from implicit.evaluation import train_test_split, precision_at_k, AUC_at_k

## 2. Data inladen

In [3]:
load_dotenv()
DB_URL = os.getenv("DB_URL")
engine = create_engine(DB_URL)

try:
    connection = engine.connect()
    print("Successfully connected to the database")
except Exception as e:
    print(f"Failed to connect to the database: {e}")

print(f"SQLAlchemy version: {sqlalchemy.__version__}")

Successfully connected to the database
SQLAlchemy version: 2.0.21


In [4]:
# materialized view
query = text('SELECT * FROM epic_5')

try:
    df = pd.read_sql_query(query, connection)
except Exception as e:
    print(f"Failed to execute query: {e}")

df.head()

,PersoonId,CampagneId,aantal_sessies,aantal_bezoeken,SessieThema,SoortCampagne,TypeCampagne,ThemaDuurzaamheid,ThemaFinancieelFiscaal,ThemaInnovatie,ThemaInternationaalOndernemen,ThemaMobiliteit,ThemaOmgeving,ThemaSalesMarketingCommunicatie,ThemaStrategieEnAlgemeenManagement,ThemaTalent,ThemaWelzijn
0,0BC3122A-4CD3-E711-80EC-001DD8B72B62,B83CFC05-E310-EE11-8F6D-6045BD895B5A,1,0,Duurzaam Ondernemen,Offline,Opleiding,0,0,0,0,0,0,0,0,0,0
1,A4701FA6-0868-E111-A00F-00505680000A,BDA1703D-FD3D-EB11-8116-001DD8B72B61,6,0,"Digitalisering, IT & Technologie",Offline,Opleiding,1,1,1,0,1,1,0,0,1,1
2,9F63D379-89E8-EC11-BB3C-00224880A0B1,F143DEBC-10D2-EC11-A7B5-000D3A4800DE,13,2,Algemeen Management,Offline,Project,0,0,0,0,0,0,0,0,0,0
3,F698B5D2-E567-E111-A00F-00505680000A,088E5178-F5C8-EB11-8123-001DD8B72B62,1,0,Internationaal Ondernemen,Offline,Netwerkevenement,0,0,0,0,0,0,0,0,0,0
4,40456E39-2206-EC11-8126-001DD8B72B62,2FB69A65-AAD8-EB11-8121-001DD8B72B61,1,0,Duurzaam Ondernemen,Offline,Netwerkevenement,0,0,0,0,0,0,0,0,0,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52567 entries, 0 to 52566
Data columns (total 17 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   PersoonId                           52567 non-null  object
 1   CampagneId                          52567 non-null  object
 2   aantal_sessies                      52567 non-null  int64 
 3   aantal_bezoeken                     52567 non-null  int64 
 4   SessieThema                         52567 non-null  object
 5   SoortCampagne                       52567 non-null  object
 6   TypeCampagne                        52567 non-null  object
 7   ThemaDuurzaamheid                   52567 non-null  int64 
 8   ThemaFinancieelFiscaal              52567 non-null  int64 
 9   ThemaInnovatie                      52567 non-null  int64 
 10  ThemaInternationaalOndernemen       52567 non-null  int64 
 11  ThemaMobiliteit                     52567 non-null  in

## 3. Content-based filtering

In [6]:
# tekstkenmerken van campagne sessies verzamelen
df_keywords = pd.DataFrame()
df_keywords['CampagneSessieInformatie'] = df['SessieThema'] + ' ' + df['SoortCampagne'] + ' ' + df['TypeCampagne']
df_keywords.head()

,CampagneSessieInformatie
0,Duurzaam Ondernemen Offline Opleiding
1,"Digitalisering, IT & Technologie Offline Oplei..."
2,Algemeen Management Offline Project
3,Internationaal Ondernemen Offline Netwerkevene...
4,Duurzaam Ondernemen Offline Netwerkevenement


In [7]:
# sentence embedding
model_keywords = SentenceTransformer('all-MiniLM-L6-v2')
keyword_vectors = model_keywords.encode(df_keywords['CampagneSessieInformatie'].tolist())

In [8]:
# matrix die toont goed campagne sessies op elkaar lijken op basis van de tekstkenmerken
similarity_matrix = np.dot(keyword_vectors, keyword_vectors.T)
print(similarity_matrix)

[[1.0000001  0.5195016  0.52811676 ... 0.512068   0.5920942  0.5840615 ]
 [0.5195016  1.0000002  0.4008546  ... 0.93616265 0.4170084  0.49040234]
 [0.52811676 0.4008546  1.         ... 0.40067333 0.8139919  0.4653141 ]
 ...
 [0.512068   0.93616265 0.40067333 ... 1.0000004  0.41431743 0.6827707 ]
 [0.5920942  0.4170084  0.8139919  ... 0.41431743 0.99999976 0.45535433]
 [0.5840615  0.49040234 0.4653141  ... 0.6827707  0.45535433 1.0000001 ]]


## 4. Collaborative filtering

SOURCE: https://benfred.github.io/implicit/index.html (Alternating Least Squares)

In [9]:
# toch groeperen op aantal sessies per persoon per campagne, geen scheiding tussen sessiethema
df = df.groupby(['PersoonId', 'CampagneId'])['aantal_sessies'].sum().reset_index()

# persoonid en campagneid omzetten naar integers voor de sparse matrix invoer van het model
persoon_mapping = {id: i for i, id in enumerate(df['PersoonId'].unique())}
campagne_mapping = {id: i for i, id in enumerate(df['CampagneId'].unique())}

# reverse mapping om later de integer terug naar de string te krijgen
persoon_reverse_mapping = {i: id for id, i in persoon_mapping.items()}
campagne_reverse_mapping = {i: id for id, i in campagne_mapping.items()}

# mapping toepassen op dataframe
df['PersoonId'] = df['PersoonId'].map(persoon_mapping)
df['CampagneId'] = df['CampagneId'].map(campagne_mapping)

In [10]:
df.head()

,PersoonId,CampagneId,aantal_sessies
0,0,0,1
1,0,1,1
2,1,2,1
3,2,3,1
4,3,4,1


In [11]:
# sparse matrix maken met aantal sessies als waarden
campagne_persoon_sessies = coo_matrix((df['aantal_sessies'], (df['CampagneId'], df['PersoonId'])))

In [12]:
# matrix wegen, om hoge aantal sessies minder zwaar te laten wegen
campagne_persoon_sessies = bm25_weight(campagne_persoon_sessies, K1=100, B=0.5)

# matrix transponeren omdat implicit werkt met (user, item) in plaats van (item, user)
persoon_sessies = campagne_persoon_sessies.T.tocsr()

In [13]:
# trainset en testset maken via de implicit library
trainset, testset = train_test_split(persoon_sessies, train_percentage=0.8, random_state=42)

In [14]:
# nodig voor de ALS berekening
os.environ["OPENBLAS_NUM_THREADS"] = "1"

In [15]:
# ALS model trainen
model = AlternatingLeastSquares(factors=128, regularization=0.15, alpha=5.0, iterations=100, random_state=42)
model.fit(trainset)

c:\Users\hanne\OneDrive\Openbaar\Documenten\TI(schoolmap)\Data Engineering Project II\DEP2-GROEP-1\venv\lib\site-packages\implicit\cpu\als.py:95: RuntimeWarning: OpenBLAS is configured to use 12 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
100%|██████████| 100/100 [00:59<00:00,  1.68it/s]


In [16]:
# ALS model evalueren
precision = precision_at_k(model, trainset, testset, K=10, num_threads=4)
auc = a = AUC_at_k(model, trainset, testset, K=10, num_threads=4)

print(f"Precision: {precision}", f"AUC: {auc}", sep="\n")

  0%|          | 0/6023 [00:00<?, ?it/s]

100%|██████████| 6023/6023 [00:00<00:00, 9160.90it/s]

Precision: 0.29319591965506364
AUC: 0.6329666986372277


## 5. ALS model functies

In [17]:
# geef gelijkaardige campagnes
def get_similar_items(item_id):
    similar = model.similar_items(item_id, 10+1) 
    return similar[0][1:]

# get_similar_items(5)

In [18]:
# geef gelijkaardige personen
def get_similar_users(user_id):
    similar = model.similar_users(user_id, 10+1) 
    return similar[0][1:]
    
# get_similar_users(5)

In [19]:
# geef recommendaties
def ALS_recommend(user_id):
    item_ids, scores = model.recommend(user_id, persoon_sessies[user_id], N=20, filter_already_liked_items=False)
    return item_ids

# ALS_recommend(5)

## 6. Hybrid model

Deze functie geeft een lijst van gebruikers terug die mogelijk geïnteresseerd zijn in een specifieke campagne. Het baseert de interesse van gebruikers op de gelijkenis tussen de opgegeven campagne en andere campagnes, evenals de aanbevelingen van het ALS-model voor die gebruikers.

Parameters:
- campaign_id (str): De unieke identificatie van de campagne waarvoor we geïnteresseerde gebruikers willen vinden.

Returns:
- top_user_ids (list): Een lijst van de top 20 gebruikers die mogelijk geïnteresseerd zijn in de opgegeven campagne.

In [24]:
def get_interested_users_hybrid(campaign_id):
    
    # nagaan of de campagne bestaat
    try:
        # map de campagne van string naar integer
        campaign_index = campagne_mapping[campaign_id]
    except KeyError:
        raise ValueError(f"Invalid campaign ID: {campaign_id}")
    
    # gelijkenis scores uit de similarity matrix van de campagne met andere campagnes
    similarity_scores = list(enumerate(similarity_matrix[campaign_index]))
    # sorteer de scores van hoog naar laag
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

    # neem de top 5 scores
    similar_campaign_indices = [i for i, _ in similarity_scores[:5]]

    interested_users = []
    
    print("Finding users...")

    # voor elke van de top 5 gelijkaardige campagnes, neem de personen die het meest geinteresseerd zijn
    for similar_campaign_index in similar_campaign_indices:
        for user_index in persoon_mapping.values():
            # verkrijg de recommended campagnes voor deze persoon
            recommended_items = ALS_recommend(user_index)
            # als de campagne in de recommended campagnes zit, voeg deze persoon toe aan de lijst
            if similar_campaign_index in recommended_items:
                interested_users.append(user_index)
    
    print("Users processing complete!")
    
    # tel de frequentie van de personen
    user_counter = Counter(interested_users)

    # geef de 20 meest voorkomende personen
    top_user_indices = [user_index for user_index, _ in user_counter.most_common(20)]

    # map de personen van integer naar string
    top_user_ids = [persoon_reverse_mapping[user_index] for user_index in top_user_indices]

    return top_user_ids

print(get_interested_users_hybrid('8FCA1D31-1EB7-E811-80F4-001DD8B72B62')) # duurt ~ 15 seconden

Finding users...
Users processing complete!
['000C22DB-55A6-EB11-811A-001DD8B72B61', '001A09BC-F067-E111-A00F-00505680000A', '007AF1AA-29C7-EA11-8113-001DD8B72B62', '00BB174E-4443-E611-80D6-005056B06EC4', '00E16E1F-B017-E811-80EF-001DD8B72B61', '012DF80F-7394-EC11-B400-000D3A2B1506', '015F36A3-5C64-EB11-811B-001DD8B72B62', '017DF258-2E6A-E811-80F2-001DD8B72B61', '018330B4-332B-EC11-8124-001DD8B72B61', '01967333-AF67-E111-A00F-00505680000A', '019BCC3A-5FA7-EB11-811E-001DD8B72B62', '01A0A460-10C2-EB11-8123-001DD8B72B62', '01AC5F2A-32F7-EA11-8115-001DD8B72B62', '01C0AECB-4510-EA11-8107-001DD8B72B62', '020340E9-0C05-EE11-8F6E-6045BD8959F5', '02325D3D-2F01-EE11-8F6E-6045BD895554', '02570CA9-6569-E111-B43A-00505680000A', '029CD013-AE67-E111-A00F-00505680000A', '02AF9DB6-EE67-E111-A00F-00505680000A', '02C5FCF6-3947-EA11-810B-001DD8B72B61']


## 7. Puur ALS model

In [28]:
def get_interested_users_ALS(item_id):
    # map de campagne van string naar integer
    item_index = campagne_mapping[item_id]
    
    # bereken de geinteresseerde personen die nog niet aan de campagne hebben deelgenomen
    item_ids, _ = model.recommend(item_index, persoon_sessies.T, N=30, filter_already_liked_items=False)
    
    # map de personen van integer naar string
    interested_users_str = [persoon_reverse_mapping[user_index] for user_index in item_ids]
    
    return interested_users_str

print(get_interested_users_ALS('8FCA1D31-1EB7-E811-80F4-001DD8B72B62')) # instant resultaat

['0C3842C4-9570-ED11-9561-6045BD895CDC', '00C5A1DD-EF67-E111-A00F-00505680000A', '13B24C89-21B7-E111-A45C-00505680000A', '010BA501-7812-EA11-8108-001DD8B72B61', '0AF32AC2-CAB1-E811-80F4-001DD8B72B62', '056A087A-DE78-E611-80E3-001DD8B72B62', '0F3EF0B1-F667-E111-A00F-00505680000A', '0446E3D7-AD67-E111-A00F-00505680000A', '0835B897-40E1-E511-A838-005056B06EB4', '08CD3491-EC9A-ED11-AAD1-6045BD895CDC', '019C790C-8AB1-E211-A85C-005056B06EC4', '090DBA8A-0530-ED11-9DB0-000D3A4673FC', '0919216F-B267-E111-A00F-00505680000A', '08F51915-5175-E911-80FE-001DD8B72B62', '0152AEAC-198E-EB11-811E-001DD8B72B62', '056DE4A3-A2D9-E711-80EE-001DD8B72B61', '08FA751C-3D48-EC11-8C62-6045BD8D2834', '0C8B21E4-C0EF-ED11-8849-6045BD895233', '06375120-5069-E111-B43A-00505680000A', '0D924825-A9D5-EC11-A7B5-000D3ABD1F85', '0EA05D3F-D2D0-EA11-8113-001DD8B72B62', '092058E4-A569-ED11-9561-6045BD895B5A', '04D205DC-4763-E411-8BF8-005056B06EC4', '01A9DD01-AE67-E111-A00F-00505680000A', '044900AA-7069-E111-B43A-00505680000A',